### GMM but multi-dimensional
with theo's help, close to running

Notes: 
I shouldn't use numpyro.param unless I am making my own guide, and I am 

In [1]:
import os
from collections import defaultdict
import torch
import numpy as np
import pandas as pd
import scipy.stats
from torch.distributions import constraints
from matplotlib import pyplot
%matplotlib inline

import pyro
import pyro.distributions as dist
from pyro import poutine
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
from pyro.infer import SVI, TraceEnum_ELBO, config_enumerate, infer_discrete

smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.8.1')

In [2]:
df = pd.read_csv('/Users/victorlhoste/Desktop/Local_PhD/NHANES/Data/data_pyro.csv', index_col=0)
df = df[["height","bmi","WHtR","hba1c","hdl","non_hdl","sbp","dbp","eGFR"]]
d=9 # dimension
df_centered = df.apply(lambda x: (x - x.mean())/x.std() )
#print(df_centered)
# data=torch.tensor(df_centered.values)
data=df_centered
data.shape

(58876, 9)

In [3]:
K=5 # Fix the number of components
h=9 # Fix the dimension
@config_enumerate
def model(data=None):
    # Global variables.
    weights = pyro.sample('weights', dist.Dirichlet(0.5 * np.ones(K)))
    scale = pyro.sample('scale', dist.HalfNormal(2.)) # same variance paramter for all clusters
    with pyro.plate('components', K):
        locs=pyro.sample('locs',dist.MultivariateNormal(np.zeros(h),np.eye(h))) #  weird

    with pyro.plate('data', len(data)):
        # Local variables
        assignment = pyro.sample('assignment', dist.Categorical(weights))
        pyro.sample('obs', dist.MultivariateNormal(locs[assignment], scale*np.eye(h)), obs=data)

In [4]:
data= np.asarray(data)

In [5]:

optim = pyro.optim.Adam({'lr': 0.1, 'betas': [0.8, 0.99]})
elbo = TraceEnum_ELBO(max_plate_nesting=1)

In [6]:
def init_loc_fn(site):
    if site["name"] == "weights":
        # Initialize weights to uniform.
        return torch.ones(K) / K
    if site["name"] == "scale":
        return (data.var() / 2).sqrt()
    if site["name"] == "locs":
        return data[torch.multinomial(torch.ones(len(data)) / len(data), K)]
    raise ValueError(site["name"])



global_guide = AutoDelta(poutine.block(model, expose=['weights', 'locs', 'scale']),
                             init_loc_fn=init_loc_fn)
svi = SVI(model, global_guide, optim, loss=elbo)

In [7]:
data=torch.from_numpy(data)
svi.loss(model, global_guide, data)

AttributeError: 'numpy.ndarray' object has no attribute 'dim'